Allows the notebook to load your username and password from the .env file.

In [ ]:
%load_ext dotenv
%dotenv

### Import dependancies and connect to peloton.

You should see a 200 status if it's connected successfully.

In [ ]:
import requests
import pandas as pd
import json
import os
import plotly.express as px
from datetime import datetime

s = requests.Session()
payload = {'username_or_email': os.getenv("peloton-username"), 'password': os.getenv("peloton-password")}
s.post('https://api.onepeloton.com/auth/login', json=payload)

## Load user data from api

In [ ]:
url = 'https://api.onepeloton.com/api/me'
# getting data from source
data = s.get(url).json()

pretty_data = json.dumps(data, indent=4)

print(pretty_data)

Show your peloton_id.

In [ ]:
peloton_id = data['id']
print(peloton_id)

Build the core DataFrame from workoutdata.

In [ ]:
df = pd.DataFrame(columns=["percentile", 
                            "workout_date", 
                            "difficulty_rating_avg", 
                            "duration_mins", 
                            "class_length",
                            "total_work"])

races = 50


def get_workout(workout_id: str):
    url = 'https://api.onepeloton.com/api/workout/' + workout_id
    # getting data from source
    data = s.get(url).json()
    
    leaderboard_rank = data['leaderboard_rank']
    total_leaderboard_users= data['total_leaderboard_users']
    avg_pos = (leaderboard_rank / total_leaderboard_users)
    timestamp = datetime.fromtimestamp(data['created_at'])
    difficulty_rating_avg = data['ride']['difficulty_rating_avg']
    pdd = (data['ride']['pedaling_duration'] / 60)
    total_work = data['total_work']/ 1000
    new_row = pd.Series({"percentile": avg_pos, 
                         "workout_date": timestamp, 
                         "difficulty_rating_avg": difficulty_rating_avg, 
                         "duration_mins": pdd, 
                         "class_length": '{0} mins'.format(int(pdd)),
                         "total_work": total_work})
    global df
    df = df.append(new_row, ignore_index=True)


def get_session_data(session_type: str):
    for w in get_data.json()['data']:
        if w['ride']['difficulty_level'] !=  "beginner":
            if w['fitness_discipline'] == session_type:
                get_workout(w['id'])
 

url = 'https://api.onepeloton.com/api/user/{0}/workouts/?joins=ride,ride.instructor&limit={1}&page=0'.format(peloton_id, races)
# getting data from source
get_data = s.get(url)
get_session_data('cycling')


df['workout_date'] = pd.to_datetime(df['workout_date'])

In [ ]:
print(df.head())

## Build a scatter plot

In [ ]:
def plot_workout_percentile():
    fig1 = px.scatter(df, x="workout_date", 
                         y="percentile", 
                         title='Workout % position',  
                         trendline="ols", 
                         trendline_color_override="red",
                    )
    fig1.update_yaxes(range=[0, 1])
    fig1.show()

plot_workout_percentile()